В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.

Измерены следующие признаки:

* state — штат США
* account_length — длительность использования аккаунта
* area_code — деление пользователей на псевдорегионы, использующееся в телекоме
* intl_plan — подключена ли у пользователя услуга международного общения
* vmail_plan — подключена ли у пользователя услуга голосовых сообщений
* vmail_message — количество голосых сообщений, который пользователь отправил / принял
* day_calls — сколько пользователь совершил дневных звонков
* day_mins — сколько пользователь проговорил минут в течение дня
* day_charge — сколько пользователь заплатил за свою дневную активность
* eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
* night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
* intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
* custserv_calls — сколько раз пользователь позвонил в службу поддержки
* treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
* mes_estim — оценка интенсивности пользования интернет мессенджерами
* churn — результат оттока: перестал ли абонент пользоваться услугами оператора

Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции

$scipy.stats.chi2_contingency(subtable, correction=False)$

Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько достигаемых уровней значимости оказались меньше, чем $\alpha$=0.05?

In [88]:
import pandas as pd
from itertools import *
from scipy.stats import chi2_contingency, fisher_exact, pearsonr, spearmanr, shapiro

In [90]:
churn_analysis = pd.read_csv('churn_analysis.csv')
churn_analysis = churn_analysis.drop('Unnamed: 0', axis = 1)
churn = churn_analysis[churn_analysis['treatment'] == 1]
churn_analysis.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


Построим таблицы сопряженноси в цикле и сразу будем считать p-value, которые меньше 0.05

In [93]:
def subtable(state):
    true_churn = [len(churn[churn['churn'] == 'True.'][churn['state']== state[0]]),
                  len(churn[churn['churn'] == 'True.'][churn['state']== state[1]])]
    false_churn = [len(churn[churn['churn'] == 'False.'][churn['state']== state[0]]),
                  len(churn[churn['churn'] == 'False.'][churn['state']== state[1]])]
    stable = pd.DataFrame(columns = ['True.', 'False.'])
    stable.loc[0] = [true_churn[0], false_churn[0]]
    stable.loc[1] = [true_churn[1], false_churn[1]]
    
    return stable

In [118]:
p_value = []
summ_p = 0
for state_comb in combinations(list(churn.state.unique()), 2):
    _, p_val, _, _ = chi2_contingency(subtable(state_comb), correction = False)
    summ_p += p_val
    if p_val < 0.05:
        p_value.append(p_val)
        
print('avg p_val = ', summ_p/len(list(combinations(list(churn.state.unique()), 2))))

<ipython-input-93-6a9246933e9a>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  true_churn = [len(churn[churn['churn'] == 'True.'][churn['state']== state[0]]),
<ipython-input-93-6a9246933e9a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['churn'] == 'True.'][churn['state']== state[1]])]
<ipython-input-93-6a9246933e9a>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  false_churn = [len(churn[churn['churn'] == 'False.'][churn['state']== state[0]]),
<ipython-input-93-6a9246933e9a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['churn'] == 'False.'][churn['state']== state[1]])]


avg p_val =  0.501827379873916


In [95]:
len(p_value)

34

3.
В основе критерия xи-квадрат лежит предположение о том, что если верна нулевая гипотеза, то дискретное биномиальное распределение данных по клеткам в таблице сопряженности может быть аппроксимировано с помощью непрерывного распределения xи-квадрат. Однако точность такой аппроксимации существенно зависит от суммарного количества наблюдений и их распределения в этой таблице (отсюда и ограничения при использовании критерия xи-квадрат).

Одним из способов коррекции точности аппроксимации является поправка Йетса на непрерывность. Эта поправка заключается в вычитании константы 0.5 из каждого модуля разности наблюденного $O_i$ и ожидаемого $E_i$ значений, то есть, статистика с такой поправкой выглядит так:

$$ \chi_{Yates}^{2} = \sum_{i=1}^{N}\frac{(\left |O_{i} -E_{i} \right | - 0.5)^{2}}{E_{i}} $$

Такая поправка, как несложно догадаться по формуле, как правило, уменьшает значение статистики $\chi^{2}$, то есть увеличивает достигаемый уровень значимости.

Эта поправка обычно используется для таблиц сопряженности размером 2x2 и для небольшого количества наблюдений. Такая поправка, однако, не является серебрянной пулей, и часто критикуется за то, что статистический критерий при ее использовании становится слишком консервативным, то есть часто не отвергает нулевую гипотезу там, где она неверна (совершает ошибку II рода).

Полезно знать, что эта поправка часто включена по умолчанию (например, в функции scipy.stats.chi2_contingency) и понимать ее влияние на оценку достигаемого уровня значимости.

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией

$scipy.stats.chi2_contingency(subtable, correction=True)$

и сравните полученные результаты, отметив все верные варианты.

In [119]:
p_value = []
summ_p = 0
for state_comb in combinations(list(churn.state.unique()), 2):
    _, p_val, _, _ = chi2_contingency(subtable(state_comb), correction = True)
    summ_p += p_val
    if p_val < 0.05:
        p_value.append(p_val)
        
print('avg p_val = ', summ_p/len(list(combinations(list(churn.state.unique()), 2))))

<ipython-input-93-6a9246933e9a>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  true_churn = [len(churn[churn['churn'] == 'True.'][churn['state']== state[0]]),
<ipython-input-93-6a9246933e9a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['churn'] == 'True.'][churn['state']== state[1]])]
<ipython-input-93-6a9246933e9a>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  false_churn = [len(churn[churn['churn'] == 'False.'][churn['state']== state[0]]),
<ipython-input-93-6a9246933e9a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['churn'] == 'False.'][churn['state']== state[1]])]


avg p_val =  0.6640566382051037


In [97]:
len(p_value)

0

4.
Что если у нас мало данных, мы не хотим использовать аппроксимацию дискретного распределения непрерывным и использовать сомнительную поправку, предположения критерия xи-квадрат не выполняются, а проверить гипотезу о том, что данные принадлежат одному распределению, нужно ?

В таком случае прибегают к так называемому точному критерию Фишера. Этот критерий не использует приближений и в точности вычисляет значение достигаемого уровня значимости используя комбинаторный подход.

Пусть у нас есть таблица сопряженности 2x2:

In [2]:
examp = pd.DataFrame({'  ': ['Воздействие 1', 'Воздействие 2', 'Сумма'],
                     'Группа 1': ['a', 'b', 'a + b'],
                     'Группа 2': ['c', 'd', 'c + d'],
                     'Сумма': ['a + c', 'b + d', 'n = a + b + c + d']})
examp

,,Группа 1,Группа 2,Сумма
0,Воздействие 1,a,c,a + c
1,Воздействие 2,b,d,b + d
2,Сумма,a + b,c + d,n = a + b + c + d


Тогда вероятность получить именно такие a, b, c, d при фиксированных значениях сумм по строкам и по столбцам) задается выражением

$$ p = \frac{\binom{a + b}{a} \binom{c + d}{c}}{\binom{n}{a + c}} = \frac{(a + b)!(c + d)!(a + c)!(b + d)!}{a! b! c! d! n!} $$

В числителе этой дроби стоит суммарное количество способов выбрать $a$ и $c$ из $a +b$ и $c + d$ соответственно. А в знаменателе — количество способов выбрать число объектов, равное сумме элементов первого столбца $a + c$ из общего количества рассматриваемых объектов $n$.

Чтобы посчитать достигаемый уровень значимости критерия Фишера, нужно перебрать все возможные значения $a$, $b$, $c$, $d$, в клетках этой таблицы так, чтобы построковые и постолбцовые суммы не изменились. Для каждого такого набора $a$, $b$, $c$, $d$ нужно вычислить значение $p_i$ по формуле выше и просуммировать все такие значения $p_i$, которые меньше или равны $p$, которое мы вычислили по наблюдаемым значениям $a$, $b$, $c$, $d$.

Понятно, что такой критерий вычислительно неудобен в силу большого количества факториалов в формуле выше. То есть даже при небольших выборках для вычисления значения этого критерия приходится оперировать очень большими числами. Поэтому данным критерием пользуются обычно только для таблиц 2x2, но сам критерий никак не ограничен количеством строк и столбцов, и его можно построить для любой таблицы $n\times m$.

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

Точный критерий Фишера удобно вычислять с помощью функции

$scipy.stats.fisher_exact$

которая принимает на вход таблицу сопряженности 2x2.

In [117]:
p_value = []
summ_p = 0
for state_comb in combinations(list(churn.state.unique()), 2):
    _, p_val = fisher_exact(subtable(state_comb))
    summ_p += p_val
    if p_val < 0.05:
        p_value.append(p_val)
        
print('avg p_val = ', summ_p/len(list(combinations(list(churn.state.unique()), 2))))

<ipython-input-93-6a9246933e9a>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  true_churn = [len(churn[churn['churn'] == 'True.'][churn['state']== state[0]]),
<ipython-input-93-6a9246933e9a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['churn'] == 'True.'][churn['state']== state[1]])]
<ipython-input-93-6a9246933e9a>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  false_churn = [len(churn[churn['churn'] == 'False.'][churn['state']== state[0]]),
<ipython-input-93-6a9246933e9a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['churn'] == 'False.'][churn['state']== state[1]])]


avg p_val =  0.6483383060020688


In [99]:
len(p_value)

10

5.
Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике.

Рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость.

In [89]:
stat, p = shapiro(churn_analysis['day_calls']) # тест Шапиро-Уилк на нормальность распределения
print('day_calls statistics=%.3f, p-value=%.3f' % (stat, p))
stat, p = shapiro(churn_analysis['mes_estim'])
print('mes_estim statistics=%.3f, p-value=%.3f' % (stat, p))

day_calls statistics=0.998, p-value=0.000
mes_estim statistics=0.999, p-value=0.078


In [85]:
corr, pval = pearsonr(churn_analysis['day_calls'], churn_analysis['mes_estim'])
corr, pval

(-0.05179435058757295, 0.002779883686972572)

6.
Еще раз рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость.

In [87]:
corr, pval = spearmanr(churn_analysis['day_calls'], churn_analysis['mes_estim'])
corr, pval

(0.043349880533927444, 0.012317367189170541)

8.
Посчитайте значение коэффицента корреляции Крамера между двумя признаками: штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

In [115]:
chi_stat = []
temp_table = pd.DataFrame(columns = ['state', 'churn_true', 'churn_false'])

for i, state in enumerate(churn.state.unique()):
    temp_table.loc[i] = [state,
                         len(churn[churn['state'] == state][churn['churn'] == 'True.']),
                         len(churn[churn['state'] == state][churn['churn'] == 'False.'])]

temp_table = temp_table.set_index('state')
stat, p_value, _, _ = chi2_contingency(temp_table)

stat, p_value

<ipython-input-115-19459f6e64ab>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['state'] == state][churn['churn'] == 'True.']),
<ipython-input-115-19459f6e64ab>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(churn[churn['state'] == state][churn['churn'] == 'False.'])]


(44.052712366925476, 0.7097590042778475)

In [113]:
len(temp_table)

51

In [114]:
v_kramer = (stat**2 / sum(temp_table.sum()) / (len(temp_table) - 1))**0.5
v_kramer

0.18809806917122976

9.
Вы прослушали большой курс и к текущему моменту обладете достаточными знаниями, чтобы попытаться самостоятельно выбрать нужный метод / инструмент / статистический критерий и сделать правильное заключение.

В этой части задания вам нужно будет самостоятельно решить, с помощью каких методов можно провести анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1).

Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

Не забудьте про поправку на множественную проверку! И не пользуйтесь односторонними альтернативами, поскольку вы не знаете, к каким действительно последствиям приводят тестируемые методы (treatment = 0, treatment = 2) !